# core

> Helper Functions for the analysis

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import subprocess
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

from pycirclize import Circos
from pycirclize.parser import Gff
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
#| export

def count_variants(vcf_file):
    """Count the number of variants in a VCF file using subprocess."""
    if vcf_file.endswith('.gz'):
        cmd = f"bcftools view -H {vcf_file} | wc -l"
    else:
        cmd = f"grep -v '^#' {vcf_file} | wc -l"
    
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return int(result.stdout.strip())

def filter_variants():
    # Define input and output VCF files
    INPUT_VCF = "../data/freebayes.annotated_pc1.vcf.gz"
    QUAL_FILTERED_VCF = "../data/filtered_qual.vcf"
    DP_FILTERED_VCF = "../data/filtered_dp.vcf"
    SNP_FILTERED_VCF = "../data/filtered_snp.vcf"
    FINAL_VCF = "../data/filtered_final.vcf"
    
    print("======================================")
    print("Starting Variant Filtering Process")
    print("======================================")
    
    # Count initial number of variants
    START_COUNT = count_variants(INPUT_VCF)
    print(f"Total variants before filtering: {START_COUNT}")
    
    # Step 1: Filter out low-quality variants (QUAL < 30)
    subprocess.run(f"bcftools filter -e 'QUAL < 30' {INPUT_VCF} -o {QUAL_FILTERED_VCF}", shell=True)
    QUAL_FILTERED_COUNT = count_variants(QUAL_FILTERED_VCF)
    print(f"Stage 1: QUAL filtering: {START_COUNT - QUAL_FILTERED_COUNT} Variants removed and {QUAL_FILTERED_COUNT} variants left")

    # Step 2: Filter variants based on per-sample depth (FORMAT/DP < 10 or > 150)
    subprocess.run(f"bcftools view -i 'FMT/DP >= 30 & FMT/DP <= 150' {QUAL_FILTERED_VCF} -o {DP_FILTERED_VCF}", shell=True)
    DP_FILTERED_COUNT = count_variants(DP_FILTERED_VCF)
    print(f"Stage 2: FORMAT/DP filtering, DP >= 30 & DP <= 150: {QUAL_FILTERED_COUNT - DP_FILTERED_COUNT} Variants removed and {DP_FILTERED_COUNT} variants left")

    # Step 3: Retain SNPs and indels (Remove other variant types if any)
    subprocess.run(f"bcftools view -v snps,indels {DP_FILTERED_VCF} -o {SNP_FILTERED_VCF}", shell=True)
    SNP_FILTERED_COUNT = count_variants(SNP_FILTERED_VCF)
    print(f"Stage 3: After keeping SNPs and indels: {DP_FILTERED_COUNT - SNP_FILTERED_COUNT} Variants removed and {SNP_FILTERED_COUNT} variants left")

    # Rename final output
    os.rename(SNP_FILTERED_VCF, FINAL_VCF)
    FINAL_COUNT = count_variants(FINAL_VCF)


In [ ]:
#| export

def read_vcf(vcf_file):
    """Reads a VCF file, automatically detecting the header and using correct column names."""
    
    # Find the header line dynamically
    with open(vcf_file, "r") as f:
        for line in f:
            if line.startswith("#CHROM"):
                header = line.strip().split("\t")  # Extract column names
                break  # Stop searching after finding header
    
    # Read VCF using Pandas, skipping comment lines
    df = pd.read_csv(vcf_file, sep="\t", comment="#", header=None, names=header)
    return df


In [ ]:
#| export

def find_index(format_value, field):
    """Finds the index of a specific field (e.g., RO, AO, DP) in the FORMAT column."""
    item_list = format_value.split(':')
    return item_list.index(field) if field in item_list else None

def expand_multiallelic_variants(df_vcf):
    """
    Extracts allele counts (RO, AO, DP) for each sample from the VCF DataFrame
    and expands multi-allelic variants into separate rows.
    """

    # Extract sample names (everything after FORMAT column)
    sample_names = df_vcf.columns[9:]  # Skip CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO, FORMAT

    # Find the index positions of RO, AO, and DP in the FORMAT field
    format_example = df_vcf['FORMAT'].values[0]  # Take the first row as an example
    ro_index = find_index(format_example, 'RO')
    ao_index = find_index(format_example, 'AO')
    dp_index = find_index(format_example, 'DP')

    # Ensure indices exist
    if None in [ro_index, ao_index, dp_index]:
        raise ValueError("RO, AO, or DP field not found in FORMAT column.")

    # Initialize an empty list to store expanded rows
    expanded_rows = []

    for _, row in df_vcf.iterrows():
        # Split ALT alleles (multi-allelic sites will have multiple ALT values)
        alt_alleles = row['ALT'].split(',')
        info_type = row['INFO'].split('TYPE=')[1].split(';')[0].split(',')
        
        # Process each ALT allele separately
        for i, (alt, inty )in enumerate(zip(alt_alleles,info_type)):
            new_row = {
                "#CHROM": row["#CHROM"],
                "POS": row["POS"],
                "REF": row["REF"],
                "ALT": alt,  # Assign each alternate allele to a separate row
                "INFO_TYPE": inty
            }

            for sample in sample_names:
                # Split FORMAT fields for the sample
                sample_values = row[sample].split(':')
                
                # Extract and store RO and DP
                new_row[f"RO_{sample}"] = int(sample_values[ro_index]) if sample_values[ro_index] != '.' else 0
                new_row[f"DP_{sample}"] = int(sample_values[dp_index]) if sample_values[dp_index] != '.' else 0

                # Extract AO for the specific ALT allele
                ao_values = sample_values[ao_index].split(',')  # Multiple values for multiple ALT alleles
                new_row[f"AO_{sample}"] = int(ao_values[i]) if i < len(ao_values) and ao_values[i] != '.' else 0

            # Append expanded row
            expanded_rows.append(new_row)

    # Convert list of dictionaries into DataFrame
    expanded_df = pd.DataFrame(expanded_rows)

    return expanded_df

In [ ]:
#vcf_file = "../data/filtered_final.vcf"  
#df_vcf = read_vcf(vcf_file)
#print(f'step 1: {df_vcf.shape}')
#df_vcf=df_vcf[(df_vcf['REF']!='N')]
#print(f'step 2: {df_vcf.shape}')
#expand_multiallelic_variants(df_vcf)

step 1: (1256, 16)
step 2: (945, 16)


,#CHROM,POS,REF,ALT,INFO_TYPE,RO_M7,DP_M7,AO_M7,RO_M5,DP_M5,...,AO_M6,RO_C3,DP_C3,AO_C3,RO_C2,DP_C2,AO_C2,RO_C1,DP_C1,AO_C1
0,CM000429,60867,TAAAAAAAAAAGATAT,TAAAAAAAAAAAGATTT,complex,9,82,2,3,46,...,2,5,57,1,1,69,0,4,62,1
1,CM000429,60867,TAAAAAAAAAAGATAT,TAAAAAAAAAAAGATAT,ins,9,82,67,3,46,...,62,5,57,45,1,69,60,4,62,50
2,CM000429,60867,TAAAAAAAAAAGATAT,TAAAAAAAAAAAAGATAT,ins,9,82,2,3,46,...,1,5,57,1,1,69,4,4,62,4
3,CM000429,60889,ACCCCACT,ACCCCCACT,ins,9,90,81,1,53,...,70,5,45,40,1,68,66,4,69,62
4,CM000429,76625,A,G,snp,76,104,28,52,80,...,28,36,83,47,78,112,34,63,112,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,CM000436,1340712,ACCCCCCCCCCCCCCACCCGCTCGCGCCA,ACCCCCCCCCCCCACCCGCTCGCGCCA,del,0,23,11,0,15,...,7,1,16,4,0,19,2,2,28,4
1933,CM000436,1340712,ACCCCCCCCCCCCCCACCCGCTCGCGCCA,ACCCCCCCCCCCACCCGCTCGCGCCA,del,0,23,0,0,15,...,4,1,16,5,0,19,8,2,28,12
1934,CM000436,1340712,ACCCCCCCCCCCCCCACCCGCTCGCGCCA,ACCCCCCCCCCACCCGCTCGCGCCA,del,0,23,0,0,15,...,2,1,16,1,0,19,0,2,28,3
1935,CM000436,1340712,ACCCCCCCCCCCCCCACCCGCTCGCGCCA,ACCCCCCCCCACCCGCTCGCGCCA,del,0,23,6,0,15,...,12,1,16,4,0,19,3,2,28,2


In [ ]:
#TAAAAAAAAAA GATAT
#TAAAAAAAAAAAGATTT

In [ ]:
#| export

def compute_frequencies(df_counts):
    """
    Computes allele frequency (AF = AO / DP) for each sample in the dataset.
    """

    # Extract sample names from AO columns
    sample_names = [col.replace("AO_", "") for col in df_counts.columns if col.startswith("AO_")]

    # Create a new DataFrame to store allele frequencies
    df_af = df_counts[['#CHROM', 'POS', 'REF', 'ALT']].copy()

    for sample in sample_names:
        ao_col = f"AO_{sample}"
        dp_col = f"DP_{sample}"
        af_col = f"AF_{sample}"
        df_af[af_col] = df_counts[ao_col] / df_counts[dp_col] #(df_counts[ro_col] + df_counts[ao_col])

    return df_af


In [ ]:
#| export

def mod_hist_legend(ax, title=False):
    """
    Creates a cleaner legend for histogram plots by using line elements instead of patches.
    
    Motivation:
    - Default histogram legends show rectangle patches which can be visually distracting
    - This function creates a more elegant legend with simple lines matching histogram edge colors
    - Positions the legend outside the plot to avoid overlapping with data
    
    Parameters:
    -----------
    ax : matplotlib.axes.Axes
        The axes object containing the histogram(s)
    title : str or bool, default=False
        Optional title for the legend. If False, no title is displayed
        
    Returns:
    --------
    None - modifies the axes object in place
    """
    # Extract the current handles and labels from the plot
    handles, labels = ax.get_legend_handles_labels()
    
    # Create new line handles that match the edge colors of histogram bars
    # This produces a cleaner, more minimal legend appearance
    new_handles = [matplotlib.lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
    
    # Create the legend with custom positioning
    # - Places legend outside the plot (to the right) to avoid obscuring the data
    # - Centers the legend vertically for better visual balance
    ax.legend(handles=new_handles, 
              labels=labels, 
              title=title,
              loc='center left', 
              bbox_to_anchor=(1, 0.5))


def clean_axes(ax, offset=10):
    """
    Customizes a matplotlib axes by removing top and right spines,
    and creating a broken axis effect where x and y axes don't touch.
    
    Parameters:
    -----------
    ax : matplotlib.axes.Axes
        The axes object to customize
    offset : int, default=10
        The amount of offset/gap between the x and y axes in points
        
    Returns:
    --------
    ax : matplotlib.axes.Axes
        The same axes object, modified in place
    """
    # Remove the top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Make the remaining spines gray for a more subtle look
    ax.spines['left'].set_color('gray')
    ax.spines['bottom'].set_color('gray')
    
    # Create the broken axis effect
    # Move the bottom spine up by offset points
    #ax.spines['bottom'].set_position(('outward', offset))
    
    # Move the left spine right by offset points
    ax.spines['left'].set_position(('outward', offset))
    
    # Return the modified axes
    return ax

In [ ]:
#| export

def make_circos_plot(data):
    
    seqid2size = {
        'CM000429': 875659,
        'CM000430': 985969,
        'CM000431': 1099352,
        'CM000432': 1104417,
        'CM000433': 1080900,
        'CM000434': 1332857,
        'CM000435': 1278458,
        'CM000436': 1344712
    }

    color_dict = {'M': 'blue', 'C': 'magenta'}

    circos = Circos(seqid2size, space=3, start=-83, end=265, endspace=False)
    circos.text("C. parvum IowaII", r=5, size=18, font={'style': 'italic'})
    
    m_samples = ['AF_M7', 'AF_M5', 'AF_M6', 'AF_M4']
    c_samples = ['AF_C3', 'AF_C2', 'AF_C1']
    
    for sector in circos.sectors:
        sector.text(sector.name[-3:])
        
        m_track = sector.add_track((80, 100))
        m_track.xticks_by_interval(200000, show_label=False)
        m_track.axis()
    
        
        c_track = sector.add_track((55, 75))
        c_track.xticks_by_interval(200000, show_label=False)
        c_track.axis()
        
        # Plot scatter points for each sample group
        for sample in m_samples:
            color = color_dict[sample[3]]
            subset = data[(data['#CHROM'] == sector.name) & (data[sample] > 0)]
            m_track.scatter(
                x=subset['POS'].values,
                y=subset[sample].values,
                c=color,
                s=3,
                vmin=0,
                vmax=1,
                alpha=0.3,
            )
        
        
        for sample in c_samples:
            color = color_dict[sample[3]]
            subset = data[(data['#CHROM'] == sector.name) & (data[sample] > 0)]
            c_track.scatter(
                x=subset['POS'].values,
                y=subset[sample].values,
                c=color,
                s=3,
                vmin=0,
                vmax=1,
                alpha=0.3,
            )

        
        # Optional: Add labels 
        if sector.name == 'CM000429':
            m_track.yticks([0, 1], ["0", "1"], side="left")
            c_track.yticks([0, 1], ["0", "1"], side="left")

            
            circos.text("Cow", r=c_track.r_center, deg=-90, color="magenta")
            circos.text("Mouse", r=m_track.r_center, deg=-90, color="blue")
    
    circos.plotfig()
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()